# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras_contrib.layers.advanced_activations import SineReLU

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [5]:
X_train /= 255
X_test /= 255

In [6]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Design Neural Network architecture

In [7]:
model = Sequential()
model.add(Dense(128, input_shape = (784,)))
model.add(BatchNormalization())
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.3))

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
sine_re_lu_1 (SineReLU)      (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
sine_re_lu_2 (SineReLU)      (None, 256)               0         
__________

#### Callbacks

In [9]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-deepnet-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')
    
tensorboard = TensorBoard("../logs/deepnet-mnist-relus-native-VIII")

#### Configure model

In [10]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [11]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping])# tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 11s 209us/step - loss: 0.4019 - acc: 0.8741 - val_loss: 0.1209 - val_acc: 0.9647
Epoch 2/50
54000/54000 [==============================] - 10s 189us/step - loss: 0.2018 - acc: 0.9380 - val_loss: 0.0933 - val_acc: 0.9700
Epoch 3/50
54000/54000 [==============================] - 10s 189us/step - loss: 0.1561 - acc: 0.9515 - val_loss: 0.0818 - val_acc: 0.9752
Epoch 4/50
54000/54000 [==============================] - 11s 199us/step - loss: 0.1346 - acc: 0.9578 - val_loss: 0.0815 - val_acc: 0.9767
Epoch 5/50
54000/54000 [==============================] - 10s 194us/step - loss: 0.1184 - acc: 0.9634 - val_loss: 0.0781 - val_acc: 0.9778
Epoch 6/50
54000/54000 [==============================] - 10s 193us/step - loss: 0.1045 - acc: 0.9663 - val_loss: 0.0713 - val_acc: 0.9790
Epoch 7/50
54000/54000 [==============================] - 10s 191us/step - loss: 0.0964 - acc: 0.9695 - val_loss: 0.0

#### Test Predictions

In [12]:
saved_model = keras.models.load_model('model_output/weights-deepnet-mnist.hdf5')#, custom_objects={'SineReLU': SineReLU})
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

0.060586610261717515

#### Test Final Accuracy

In [15]:
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 1s 111us/step
Final loss: 0.0658, final accuracy: 0.9850


In [14]:
#Final loss: 0.0765, final accuracy: 0.9833 -> SineReLU (0.0083); STD loss => 0.05375531819714868
#Final loss: 0.0745, final accuracy: 0.9827 -> SineReLU (0.0080); STD loss => 0.05386899405890265
#Final loss: 0.0859, final accuracy: 0.9819 -> SineReLU (0.0085); STD loss => 0.053545432633176175
#Final loss: 0.0729, final accuracy: 0.9819 -> ReLU; STD loss => 0.057364919558005455
#Final loss: 0.0761, final accuracy: 0.9823 -> ReLU; STD loss => 0.057064730691239844
#Final loss: 0.0823, final accuracy: 0.9829 -> ReLU; STD loss => 0.05736969016884351